# ENGSCI 762 - AS1 - Part I
## Noel D'Souza - ndso092 - 449609993

## 4. Prepare the data

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import sklearn

%matplotlib inline
%load_ext autoreload
%autoreload 2
sklearn.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'0.20.3'

In [5]:
train = pd.read_csv('../data/train.csv', index_col=0, na_values='NaN', keep_default_na=False)
test = pd.read_csv('../data/test.csv', index_col=0, na_values='NaN', keep_default_na=False)
# test = pd.read_csv('../data/test.csv', index_col=0)
# train.info()

In [6]:
# # Load in the data
# def split_features_labels(filename = '../data/train.csv', 
#                           target_col='SalePrice'):
#     df = pd.read_csv(filename, index_col=0, na_values='NaN', keep_default_na=False)
#     X = df.drop(target_col, axis=1)
#     y = df[target_col].copy()
#     return X, y

# housing, y_train = split_features_labels()
# housing_test = pd.read_csv('../data/test.csv', index_col=0, na_values='NaN', keep_default_na=False)

In [7]:
housing = train.drop('SalePrice', axis=1)
y_train = train['SalePrice'].copy()

housing_test = test

In [8]:
num_cols = ['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'EnclosedPorch', 'Fireplaces',
 'FullBath', 'GrLivArea', 'HalfBath', 'KitchenAbvGr', 'LotArea', 'LowQualFinSF', 'MSSubClass',
 'MiscVal', 'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'ScreenPorch',
 'TotRmsAbvGrd', 'WoodDeckSF', 'YearBuilt', 'YearRemodAdd', 'YrSold']

In [9]:
# Get numerical features, this preprocessing strategy corresponds to slide ... of lecture 7,
# because .... 
# N: But doesn't this method of Andreas miss out like 33-25 numerical variables?
housing_num = housing[num_cols]
housing_test_num = housing_test[num_cols]

# N: With the other method, why did we need to drop "SalePrice" from the equation?
# num = train.select_dtypes(include="number").drop(['SalePrice'], axis=1)
# test_num = test[num.columns]

# housing_num.info()

In [10]:
# Get categorical features
housing_cat = housing.select_dtypes(include='object')
housing_test_cat = housing_test[housing_cat.columns]
# housing_cat.info()

### 4.1 Identify suitable transformations

#### 4.1.1 Transforming numerical features

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(housing_num)
X_test_scaled = scaler.transform(housing_test_num)

X_train_scaled.shape, X_test_scaled.shape

C:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


((1460, 25), (1459, 25))

#### 4.1.2 Decomposing categorical features

In [12]:
from sklearn.preprocessing import OneHotEncoder

# Encoding to derive the column categories based on the unique values
encoder = OneHotEncoder(handle_unknown='ignore')

X_train_cat_encoded = encoder.fit_transform(housing_cat)
X_test_cat_encoded = encoder.transform(housing_test_cat)

# The method .toarray() converts the sparse array representation into a matrix
X_train_cat_encoded = X_train_cat_encoded.toarray()
X_test_cat_encoded = X_test_cat_encoded.toarray()

X_train_cat_encoded.shape, X_test_cat_encoded.shape

((1460, 805), (1459, 805))

#### 4.1.3 Merging numerical and categorical features manually

In [13]:
# Combine numerical and categorical feature matrices for train and test set
X_train = np.c_[X_train_scaled, X_train_cat_encoded]
X_test = np.c_[X_test_scaled, X_test_cat_encoded]

X_train.shape, X_test.shape

((1460, 830), (1459, 830))

### 4.2 Create new features

In [ ]:
# Number of "full" bathrooms
train_num_bathrooms = housing.FullBath + housing.BsmtFullBath + 0.5 * housing.HalfBath + 0.5 * housing.BsmtHalfBath
# test_num_bathrooms = housing_test.FullBath + housing_test.BsmtFullBath + 0.5 * housing_test.HalfBath + 0.5 * housing_test.BsmtHalfBath

# # Proportion of total rooms that are bedrooms (above grade and excluding bathrooms)
train_proportion_bedrooms = housing.BedroomAbvGr / housing.TotRmsAbvGrd
test_proportion_bedrooms = housing_test.BedroomAbvGr / housing_test.TotRmsAbvGrd

# # Total indoor SF area
train_total_SF = housing.TotalBsmtSF + housing['1stFlrSF'] + housing['2ndFlrSF']
test_total_SF = housing_test.TotalBsmtSF #+ housing_test['1stFlrSF'] + housing_test['2ndFlrSF']

# # Total above grade living SF area as a fraction of total lot area
train_livingSF_to_lot = housing.GrLivArea / housing.LotArea
test_livingSF_to_lot = housing_test.GrLivArea / housing_test.LotArea

# # Proportion of total rooms that are kitchen(s)
train_proportion_kitchens = housing.KitchenAbvGr / housing.TotRmsAbvGrd
test_proportion_kitchens = housing_test.KitchenAbvGr / housing_test.TotRmsAbvGrd

# # Total porch and/or deck SF area
train_total_porch_SF = housing.WoodDeckSF + housing.OpenPorchSF + housing.EnclosedPorch + housing['3SsnPorch'] + housing.ScreenPorch
test_total_porch_SF = housing_test.WoodDeckSF + housing_test.OpenPorchSF + housing_test.EnclosedPorch + housing_test['3SsnPorch'] + housing_test.ScreenPorch

In [ ]:
train_bool_2ndfloor = housing['2ndFlrSF'].apply(lambda x: 1 if x>0 else 0)
test_bool_2ndfloor = housing_test['2ndFlrSF'].apply(lambda x: 1 if x>0 else 0)

train_bool_pool = housing.PoolArea.apply(lambda x: 1 if x>0 else 0)
test_bool_pool = housing_test.PoolArea.apply(lambda x: 1 if x>0 else 0)

train_bool_porch = train_total_porch_SF.apply(lambda x: 1 if x>0 else 0)
test_bool_porch = test_total_porch_SF.apply(lambda x: 1 if x>0 else 0)

train_bool_fireplace = housing.Fireplaces.apply(lambda x: 1 if x>0 else 0)
test_bool_fireplace = housing_test.Fireplaces.apply(lambda x: 1 if x>0 else 0)

train_bool_basement = housing.TotalBsmtSF.apply(lambda x: 1 if x>0 else 0)
# test_bool_basement = housing_test.TotalBsmtSF.apply(lambda x: 1 if x>0 else 0)

train_bool_garage = housing.GarageArea.apply(lambda x: 1 if x>0 else 0)
# test_bool_garage = housing_test.GarageArea.apply(lambda x: 1 if x>0 else 0)

In [13]:
X_train_final = np.c_[X_train, 
                      train_num_bathrooms,
                      train_proportion_bedrooms,
                      train_total_SF,
                      train_livingSF_to_lot,
                      train_proportion_kitchens,
                      train_total_porch_SF,
                      train_bool_2ndfloor,
                      train_bool_pool,
                      train_bool_porch,
                      train_bool_fireplace,
                      train_bool_basement,
                      train_bool_garage]
X_test_final = np.c_[X_test, 
#                      test_num_bathrooms, 
                     test_proportion_bedrooms,
                     test_total_SF,
                     test_livingSF_to_lot,
                     test_proportion_kitchens,
                     test_total_porch_SF,
                     test_bool_2ndfloor,
                     test_bool_pool,
                     test_bool_porch,
                     test_bool_fireplace]

X_train_final.shape, X_test_final.shape

((1460, 842), (1459, 839))

## 5. Evaluate three different regression algorithms

### 5.1 Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Infer param_grid from 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
# However, for simple linear regression this is kind of over-the-top.
param_grid = {'fit_intercept': [True, False],
             'normalize': [True, False]}

lin_reg = LinearRegression()
lin_search = GridSearchCV(lin_reg, param_grid, cv=10, 
                          scoring = 'neg_mean_squared_error')

In [17]:
lin_search.fit(X_train_final, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [18]:
np.sqrt(-lin_search.best_score_)

13092444412.11939

In [19]:
lin_search.best_params_

{'fit_intercept': True, 'normalize': False}

In [17]:
from sklearn.metrics import mean_squared_error

in_sample_mse = mean_squared_error(y_train, 
                           lin_search.predict(X_train_final))

NameError: name 'lin_search' is not defined

In [21]:
lin_reg_rmse = pd.Series({'in sample': np.sqrt(in_sample_mse),
                         'out of sample': np.sqrt(-lin_search.best_score_)})
lin_reg_rmse # The linear regression model is severly overfitted.

in sample        1.181393e+04
out of sample    1.309244e+10
dtype: float64

### 5.2 RandomForrest

In [14]:
# From lecture 13

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]},
              {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
             ]
forest_reg = RandomForestRegressor()
rfr_search = GridSearchCV(forest_reg, param_grid, cv=10, scoring='neg_mean_squared_error')

rfr_search.fit(X_train_final, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [15]:
# At least one of the best performing parameters is at the boundary of the param_grid,
# therefore you should increase the respective search region
rfr_search.best_params_ 

{'max_features': 8, 'n_estimators': 30}

In [16]:
rfr_in_sample_mse = mean_squared_error(y_train, 
                           rfr_search.predict(X_train_final))

NameError: name 'mean_squared_error' is not defined

In [30]:
# Out-of-sample
np.sqrt(-rfr_search.best_score_)

35977.51549747917

In [31]:
rfr_reg_rmse = pd.Series({'in sample': np.sqrt(rfr_in_sample_mse),
                         'out of sample': np.sqrt(-rfr_search.best_score_)})
rfr_reg_rmse 

in sample        12846.805055
out of sample    35977.515497
dtype: float64

### 5.3 Support Vector Machines

#### 5.3.1 Support Vector Regression

In [14]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

svr = LinearSVR(max_iter=100000)
svr.fit(X_train_final, y_train)

svr_in_sample_mse = mean_squared_error(y_train,
                                      svr.predict(X_train_final))

In [15]:
svr_reg_rmse = pd.Series({'in sample': np.sqrt(svr_in_sample_mse),
                         'out of sample': np.nan})
svr_reg_rmse 

in sample        47497.630128
out of sample             NaN
dtype: float64

In [16]:
rmse = pd.DataFrame({'LinReg': lin_reg_rmse,
                    'RFR': rfr_reg_rmse,
                    'SVR': svr_reg_rmse}
                   )
rmse

NameError: name 'lin_reg_rmse' is not defined

#### 5.3.2 Grid Search

In [32]:
X_test_final = np.nan_to_num(X_test_final)

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# The following parameter grid has been adapted from 
# https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html
param_grid = [dict(kernel=['rbf'], gamma=[0.1], 
               epsilon=[0.1]),
              dict(kernel=['linear'], gamma=['auto']),
              dict(kernel=['poly'], gamma=['auto'], degree=[3], 
                   epsilon=[.1])
             ]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_final_mm = scaler.fit_transform(X_train_final)
X_test_final_mm = scaler.transform(X_test_final)

sv_reg = SVR(max_iter=100000, C=100)

svr_search = GridSearchCV(sv_reg, param_grid, 
                          cv=10, scoring='neg_mean_squared_error')

svr_search.fit(X_train_final_mm, y_train)

ValueError: could not convert string to float: 'NA'

In [ ]:
svr_search.best_params_

In [94]:
# Out-of-sample
np.sqrt(-svr_search.best_score_)

NameError: name 'svr_search' is not defined

In [ ]:
svr_in_sample_mse = mean_squared_error(y_train, 
                           svr_search.predict(X_train_final))

In [ ]:
svr_reg_rmse = pd.Series({'in sample': np.sqrt(svr_in_sample_mse),
                         'out of sample': np.sqrt(-svr_search.best_score_)})
svr_reg_rmse 

In [ ]:
rmse = pd.DataFrame({'LinReg': lin_reg_rmse,
                    'RFR': rfr_reg_rmse,
                    'SVR': svr_reg_rmse}
                   )
rmse

## 6. Submit predictions to Kaggle

In [92]:
y_test = pd.Series(rfr_search.predict(X_test_final),
                   index = housing_test.index,
                  name='SalePrice')
y_test.head()

ValueError: could not convert string to float: 'NA'

In [ ]:
# y_test.to_csv('prediction.csv',
#              header=True)